In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
#ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U 해결
#! pip install -U accelerate
#! pip install -U transformers

#### 허깅페이스 트랜스포머 활용 라이브러리사용 시 
!pip install transformers==4.40.1 datasets==2.19.0 huggingface_hub==0.23.0 -qq

- huggingface transformer 라이브러리 사용하면 허깅페이스 모델 허브의 모델을 쉽게 불러와 사용가능
- **꼭 알아야 하는 사실은 huggingface는 모델을 body와 head로 구분한다. 이유는 같은 바디를 사용하면서 다른 작업에 사용할 수 있도록 만들기 위해서다**
- 모델의 바디만 불러올 수 있고, 헤드와 함께 불러올 수도 있다.
- 허깅페이스 모델을 저장할 때 config.json 파일이 함께 저장되는데,해당 설정 파일에는 **모델의 종류,설정 파라미터,어휘 사전 크기,토크나이저 클래스 등**이 저장된다. 이를 참고해서 AutoModel,AutoTokenizer 는 적절한 모델과 토크나이저를 불러옴

In [4]:
## 모델 바디 불러오기
from transformers import AutoModel
model_id = 'klue/roberta-base'
model= AutoModel.from_pretrained(model_id)

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AutoModelForSequenceClassification은 텍스트 시퀀스 분류를 위한 헤드가 포함된 모델을 불러올 때 사용하는 클래스

In [5]:
## 텍스트 분류 헤드가 붙은 모델 불러오기
from transformers import AutoModelForSequenceClassification
model_id = 'SamLowe/roberta-base-go_emotions'
classification_model = AutoModelForSequenceClassification.from_pretrained(model_id)

### 허깅 페이스로 불러온 모델의 저장 경로
~/.cache/huggingface/transformers/

- AutoConfig 를 활용하면 모델의 특성을 확인 할 수 있음

In [6]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained("klue/roberta-base")
print(config)

RobertaConfig {
  "_name_or_path": "klue/roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizer",
  "transformers_version": "4.45.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 32000
}



### 토크나이저 
- 토크나이저는 텍스트를 토큰 단위로 나누고 각 토큰을 대응 하는 토큰 아이디로 변환하거나 필요의 경우 특수 토큰을 추가하는 역할도 한다.
- AutoTokenizer 클래스를 이용해 토크나이저를 불러올 수 있다.

In [7]:
from transformers import AutoTokenizer
model_id = 'klue/roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [8]:
#토크나이저 사용해보기
tokenized = tokenizer('토크나이저는 텍스트를 토큰 단위로 나누고 토큰 아이디로 변환하거나 특수 토큰 추가 역할을 한다')
print(tokenized)

print(tokenizer.convert_ids_to_tokens(tokenized['input_ids']))

print(tokenizer.decode(tokenized['input_ids']))

print(tokenizer.decode(tokenized['input_ids'],skip_special_tokens=True))

{'input_ids': [0, 9157, 7461, 2190, 2259, 8509, 2138, 1793, 2855, 5385, 2200, 4835, 2088, 1793, 2855, 15129, 2200, 15070, 2205, 9253, 5177, 1793, 2855, 4140, 4008, 2069, 3605, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', '토크', '##나이', '##저', '##는', '텍스트', '##를', '토', '##큰', '단위', '##로', '나누', '##고', '토', '##큰', '아이디', '##로', '변환', '##하', '##거나', '특수', '토', '##큰', '추가', '역할', '##을', '한다', '[SEP]']
[CLS] 토크나이저는 텍스트를 토큰 단위로 나누고 토큰 아이디로 변환하거나 특수 토큰 추가 역할을 한다 [SEP]
토크나이저는 텍스트를 토큰 단위로 나누고 토큰 아이디로 변환하거나 특수 토큰 추가 역할을 한다


- input_ids는 토큰화 했을 때 각 토큰이 토크나이저 사전의 몇번째 항목인지 나타낸다.
- 첫번째 항목은 0 인데 이는 [CLS] 토큰으로 대응된다.
- attention_mask가 1이면 padding token이 아닌 실제 토큰을 의미함.
- token_type_ids가 0 이면 일반적으로 첫번째 문장을 의미한다.
- 토큰 아이디를 다시 텍스트로 돌리고 싶다면 토크나이저의 decode 메서드를 사용하고, 만약 [CLS] 나 [SEP] 과 같은 특수 토큰을 제외하고싶으면 skip_special_tokens 인자를 True로 설정하면 된다.

In [9]:
# 여러개 문장 처리하기
print(tokenizer(['첫번째 문장','두번째 문장']))
# 2개의 문장이 한번에 모델에 입력되길 원하면 리스트로 묶어야
print(tokenizer([['첫번째 문장','두번째 문장']]))

{'input_ids': [[0, 1656, 2517, 3135, 6265, 2], [0, 864, 2517, 3135, 6265, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}
{'input_ids': [[0, 1656, 2517, 3135, 6265, 2, 864, 2517, 3135, 6265, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [10]:
# batch_decode() 메서드 사용 시 input_ids 토큰 아이디를 문자열로 복원 할 수 있다.
# 기본적으로 토큰화시 CLS 와 SEP으로 나뉘는데 2개의 문장 토큰화 시 SEP으로 두 문장을 구분할 수 있다.
first_tokenized_result = tokenizer(['첫번째 문장','두번째 문장'])['input_ids']
print(tokenizer.batch_decode(first_tokenized_result))

second_tokenized_result = tokenizer([['첫번째 문장','두번째 문장']])['input_ids']
print(tokenizer.batch_decode(second_tokenized_result))

['[CLS] 첫번째 문장 [SEP]', '[CLS] 두번째 문장 [SEP]']
['[CLS] 첫번째 문장 [SEP] 두번째 문장 [SEP]']


- 토큰화 결과 중 token_type_ids는 문장을 구분하는 역할. BERT는 학습 시 2개의 문장이 서로 이어지는지 맞추는 NSP(Next Sentence Prediction)작업을 활용하는데 이를 위해 문장을 구분하는 token type id를 만들었다.
- RoBERTa 계열 모델의 경우 NSP작업을 학습 과정에서 제거 했기 때문에 문장 토큰 구분이 필요없다. 

In [11]:
bert_tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
print(bert_tokenizer([['첫번째 문장','두번째문장']]))

roberta_tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')
print(roberta_tokenizer([['첫번째 문장','두번째문장']]))

en_roberta_tokenizer = AutoTokenizer.from_pretrained('roberta-base')
print(en_roberta_tokenizer([['first sentence','second sentence']]))

{'input_ids': [[2, 1656, 2517, 3135, 6265, 3, 864, 2517, 3135, 2346, 2121, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
{'input_ids': [[0, 1656, 2517, 3135, 6265, 2, 864, 2517, 3135, 2346, 2121, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
{'input_ids': [[0, 9502, 3645, 2, 2, 10815, 3645, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1]]}


- Attention_mask는 해당 토큰이 패딩인지 정보를 담고 있다.
- 패딩은 모델에 입력하는 토큰 아이디의 길이를 맞추기 위해 추가하는 특수 토큰이다
- padding 인자에 longest를 추가하면 긴 문장에 맞춰 패딩 토큰을 추가한다.

In [12]:
#attention_mask 확인
tokenizer(['짧은 문장','이건 더 긴 문장이다'],padding = 'longest')

{'input_ids': [[0, 1599, 2073, 6265, 2, 1, 1], [0, 5370, 831, 646, 6265, 28674, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1]]}

## DataSets 활용하기

In [13]:
from datasets import load_dataset
klue_mrc_dataset = load_dataset('klue','mrc')
klue_mrc_dataset

#만약에 train(test,validataion)데이터만 필요하다고 하면 아래와 같은 코드
#only_train = load_dataset('klue','mrc','split = 'train')

Generating train split:   0%|          | 0/17554 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5841 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
        num_rows: 17554
    })
    validation: Dataset({
        features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
        num_rows: 5841
    })
})

In [ ]:
# 로컬의 csv 데이터 활용하기
from datasets import load_dataset
dataset = load_dataset('csv',data_files = 'file.csv')
#딕셔너리 사용
from datasets import Dataset
my_dict = {'A' :[1,2,3]}
dataset = Dataset.from_dict(my_dict)
#판다스 활용
from datasets import Dataset
df = pd.DataFrame({'A':[1,2,3]})
dataset = Dataset.from_pandas(df)

## 모델 학습 하기

- 한국어 기사 제목으로 기사의 카테고리 분류하는 모델 만들기
- HuggingFace Transformer에서는 학습 과정을 추상화한 Trainer API를 제공한다. 이를 사용하면 학습을 간편하게 할 수 있지만,내부에서 어떤 과정을 거치는지 알기 어렵다는 단점도 있다.

In [14]:
#데이터 셋 준비
klue_tc_train = load_dataset('klue','ynat',split = 'train')
klue_tc_eval = load_dataset('klue','ynat',split = 'validation')
klue_tc_train[0]

Generating train split:   0%|          | 0/45678 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9107 [00:00<?, ? examples/s]

{'guid': 'ynat-v1_train_00000',
 'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영',
 'label': 3,
 'url': 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=227&oid=001&aid=0008508947',
 'date': '2016.06.30. 오전 10:36'}

guid : 데이터의 고유 id


title : 뉴스 제목

label : 속한 카테고리 ID

url : 뉴스 링크

date : 뉴스 입력 시간

In [15]:
klue_tc_train.features['label'].names

['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치']

In [16]:
# 불필요 행 제거
klue_tc_train = klue_tc_train.remove_columns(['guid','url','date'])
klue_tc_eval = klue_tc_eval.remove_columns(['guid','url','date'])
# 레이블 카테고리 열 추가
klue_tc_label = klue_tc_train.features['label']
def make_label(batch):
    batch['label_str'] = klue_tc_label.int2str(batch['label'])
    return batch
klue_tc_train = klue_tc_train.map(make_label,batched=True,batch_size = 1000)
klue_tc_train[0]

Map:   0%|          | 0/45678 [00:00<?, ? examples/s]

{'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영', 'label': 3, 'label_str': '생활문화'}

In [17]:
#10,000개만 가지고 해보자
train_dataset = klue_tc_train.train_test_split(test_size=10000 , shuffle = True)['test']
dataset = klue_tc_eval.train_test_split(test_size=1000, shuffle = True)
test_dataset = dataset['test']
valid_dataset = dataset['train'].train_test_split(test_size=1000,shuffle=True)['test']


## Trainer API 사용하기

- 학습에 필요한 다양한 기능(데이터로더,로깅,평가,저장)을 TrainingArguments만으로 쉽게 활용할 수 있다.

In [20]:
import torch
import numpy as np
from transformers import (Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer)
def tokenizer_function(examples):
    return tokenizer(examples['title'],padding='max_length',truncation=True)
model_id = 'klue/roberta-base'
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels = len(train_dataset.features['label'].names))
tokenizer = AutoTokenizer.from_pretrained(model_id)
train_dataset = train_dataset.map(tokenizer_function,batched=True)
valid_dataset = valid_dataset.map(tokenizer_function,batched=True)
test_dataset = test_dataset.map(tokenizer_function,batched=True)

#학습이 끝날 때마다 검증 데이터 평가가 이루어 지도록 evaluation_strategy = epoch으로 설정
#학습이 잘 이루어 지는지 확인할 때 사용할 평가지표 정의. 예측 결과인 eval_pred를 입려긍로 받아 예측 결과 중 가장 큰 값을 갖는 클래스를 np.argmax로 뽑아 predictions 변수에 저장하고 labels와 같은 값을 갖는 결과 비율을 정확도로 반환
training_args = TrainingArguments(
    output_dir= "./result",
    num_train_epochs = 1,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    evaluation_strategy = 'epoch',
    learning_rate = 5e-5,
    push_to_hub = False)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    prediction = np.argmax(logits,axis = -1)
    return {'Accuracy' : (prediction==labels).mean()}

#학습 진행하기
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
trainer.train()

trainer.evaluate(test_dataset)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

C:\Users\ktaek\anaconda3\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\ktaek\anaconda3\Lib\site-packages\transformers\models\roberta\modeling_roberta.py:370: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.569300,0.497314,0.849000


{'eval_loss': 0.5668138265609741,
 'eval_Accuracy': 0.831,
 'eval_runtime': 26.5605,
 'eval_samples_per_second': 37.65,
 'eval_steps_per_second': 4.706,
 'epoch': 1.0}

## Trainer API 없이 학습하기

In [36]:
# 학습에 사용할 모델과 토크나이저 준비
import torch
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from transformers import AdamW

# title 토큰화
def tokenize_function(examples):
    return tokenizer(examples['title'],padding='max_length',truncation=True)
#모델,토크나이저 불러오기
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_id = 'klue/roberta-base'
model = AutoModelForSequenceClassification.from_pretrained(model_id,num_labels = len(train_dataset.features['label'].names))
tokenizer = AutoTokenizer.from_pretrained(model_id)
model.to(device)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

1. model.train()으로 모델을 학습모드로 전환
2. 배치 데이터를 가지고와서 모델에 입력으로 전달(input_ids,attention_mask,정답레이블 labels)
3. 레이블과의 차이를 통해 계산된 loss값으로 역전파 수행하고, 옵티마이저의 step 호출로 역전파 결과를 바탕으로 모델 업데이트
4. total_loss의 경우 학습이 잘되고 있는지 집게

In [45]:
#학습 데이터 준비
def make_dataloader(dataset, batch_size, shuffle=True):
    dataset = dataset.map(tokenize_function,batched=True).with_format('torch')
    dataset = dataset.rename_column('label','labels')
    dataset = dataset.remove_columns(column_names = ['title'])
    return DataLoader(dataset,batch_size = batch_size, shuffle = shuffle)
train_dataloader = make_dataloader(train_dataset,batch_size = 8, shuffle=True)
valid_dataloader = make_dataloader(valid_dataset,batch_size = 8, shuffle=False)
test_dataloader = make_dataloader(test_dataset,batch_size = 8, shuffle=False)

#학습 함수
def train_epoch(model, data_loader, optimizer):
    model.train()
    total_loss = 0
    for batch in tqdm(data_loader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids,attention_mask=attention_mask,labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss +=loss.item()
    avg_loss = total_loss / len(data_loader)
    return avg_loss

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [46]:
# 평가 함수 정의
def evaluate(model,data_loader):
    model.eval()
    total_loss=0
    predictions=[]
    true_labels=[]
    with torch.no_grad():
        for batch in tqdm(data_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids,attention_mask=attention_mask,labels=labels)
            logits = outputs.logits
            loss = outputs.loss
            total_loss += loss.item()
            preds = torch.argmax(logits,dim=-1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    avg_loss = total_loss/len(data_loader)
    accuracy = np.mean(np.array(predictions) == np.array(true_labels))
    return avg_loss,accuracy
# 학습 루프 수행하기
num_epochs = 1
optimizer = AdamW(model.parameters(),lr=5e-5)

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    train_loss = train_epoch(model,train_dataloader,optimizer)
    print(f'Training loss : {train_loss}')
    valid_loss, valid_accuracy = evaluate(model, valid_dataloader)
    print(f"Validataion Loss : {valid_loss}")
    print(f"Validataion Accuracy : {valid_accuracy}")
# 테스트
_,test_accuracy = evaluate(model,test_dataloader)
print(f'Test Accuracy : {test_accuracy}')


Epoch 1/1


  0%|          | 0/1250 [00:00<?, ?it/s]

Training loss : 0.6234439497977495


  0%|          | 0/125 [00:00<?, ?it/s]

Validataion Loss : 0.573597308754921
Validataion Accuracy : 0.817


  0%|          | 0/125 [00:00<?, ?it/s]

Test Accuracy : 0.793


### 학습 모델 허깅페이스 업로드
- 업로드 방법은 크게 Trainer 사용했을 때, 사용하지 않았을 때로 나뉨
- Trainer 사용한 경우  trainer 인스턴스에서 push_to_hub() 메서드를 사용하면 학습한 모델과 토크나이저를 함께 모델 허브에 업로드한다
- 직접 학습한 경우 모델과 토크나이저를 각각 push_to_hub()로 업로드 할 수 있다.

In [49]:
from huggingface_hub import login
login(token = '---')
repo_id = f"ktaek94/roberta-base-klue-ynat-classification"
#Trainer사용시
trainer.push_to_hub(repo_id)
#직접 학습 시
model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\ktaek\.cache\huggingface\token
Login successful


model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

events.out.tfevents.1728727849.DESKTOP-340AC9K.21016.0:   0%|          | 0.00/6.40k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1728728705.DESKTOP-340AC9K.21016.1:   0%|          | 0.00/411 [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ktaek94/roberta-base-klue-ynat-classification/commit/f91083419f86288b66322d56ca66fccce74a3243', commit_message='Upload tokenizer', commit_description='', oid='f91083419f86288b66322d56ca66fccce74a3243', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ktaek94/roberta-base-klue-ynat-classification', endpoint='https://huggingface.co', repo_type='model', repo_id='ktaek94/roberta-base-klue-ynat-classification'), pr_revision=None, pr_num=None)

## 모델 추론하기
- 모델을 활용하기 쉽도록 추상화한 파이프라인을 활용하는 방법
- 직접 모델과 토크나이저를 불러와 활용하는 방법


### 파이프라인 활용한 추론
- 토크나이저와 모델을 결합해 데이터의 전후처리와 모델 추론을 간단하게 수행하는 pipeline을 제공한다.
- 파이프라인은 크게 작업 종류,모델, 설정을 입력으로 받는다. 작업 종류는 텍스트 분류,토큰 분류 등 작업에 맞춰 설정하고 모델에 저장소 아이디를 설정하면된다.
- 자세한 설명은 huggingface.co/docs/transformers/main_classes/pipelines#transformers.pipeline.config에서 확인 가능하다
- 추론하고자 하는 텍스트를 입력하면 레이블과 그 확률을 반환한다.

In [60]:
dataset['train']['title'][:5]

['무늬만 국산 OEM 수입차 판매 반등세…차종도 다양화',
 '사우디 원유 공급 부족한가…美 증산요구에 이의',
 '게시판 KBS 안전한 대한민국 주간 특별편성',
 '이언주 참전명예수당 국민연금 평균 20%까지 인상법 발의',
 '안산소식 디딤돌 일자리 사업 참여자 37명 모집']

In [61]:
from transformers import pipeline
model_id = 'ktaek94/roberta-base-klue-ynat-classification'
model_pipeline = pipeline('text-classification',model=model_id)
model_pipeline(dataset['train']['title'][:5])


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'LABEL_1', 'score': 0.9278706908226013},
 {'label': 'LABEL_4', 'score': 0.9798926115036011},
 {'label': 'LABEL_3', 'score': 0.7625777721405029},
 {'label': 'LABEL_1', 'score': 0.8380774259567261},
 {'label': 'LABEL_2', 'score': 0.781562328338623}]

### 직접 추론
- 직접 모델과 토크나이저를 불러와 추론을 구현할 수 있다.
- __init__ 메서드에서 입력받은 모델 아이디에 맞는 모델과 토크나이저를 불러온다.
- Custompipeline의 인스턴스를 호출 할때 __call__메서드를 사용하는데 tokenizer를 통해 토큰화를 수행
- 모델 추론을 수행

In [63]:
import torch
from torch.nn.functional import softmax
from transformers import AutoModelForSequenceClassification, AutoTokenizer
class CustomPipeline:
    def __init__(self,model_id):
        self.model = AutoModelForSequenceClassification.from_pretrained(model_id)
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model.eval()
    def __call__(self,texts):
        tokenized = self.tokenizer(texts, return_tensors = 'pt', padding=True, truncation=True)
        with torch.no_grad():
            outputs = self.model(**tokenized)
            logits = outputs.logits
        probabilities = softmax(logits, dim = -1)
        scores, labels = torch.max(probabilities, dim = -1)
        labels_str = [self.model.config.id2label[label_idx] for label_idx in labels.tolist()]
        return [{'label' :label,'score':score.item()} for label, score in zip(labels_str,scores)]
custom_pipeline = CustomPipeline(model_id)
custom_pipeline(dataset['train']['title'][:5])

[{'label': 'LABEL_1', 'score': 0.9278706908226013},
 {'label': 'LABEL_4', 'score': 0.9798926115036011},
 {'label': 'LABEL_3', 'score': 0.7625780701637268},
 {'label': 'LABEL_1', 'score': 0.8380774259567261},
 {'label': 'LABEL_2', 'score': 0.781562328338623}]